In [1]:
import os
import shutil
import random
from PIL import Image
import numpy as np



In [ ]:
def mask_has_content(mask_path, threshold=2):
    """Check if the mask image is non-empty."""
    if not os.path.exists(mask_path):
        return False
    # return True
    img = Image.open(mask_path).convert("L")
    return np.array(img).sum() > threshold

def prepare_classification_dataset(src_root="./..", dst_root="data_classification", train_ratio=0.8):
    os.makedirs(dst_root, exist_ok=True)
    categories = ["fire", 'road']
    for phase in ["train", "val"]:
        for category in categories:
            os.makedirs(os.path.join(dst_root, phase, category), exist_ok=True)

    # 1. Fire images
    fire_imgs = [os.path.join(src_root, "archive (6)/fire_dataset/fire_images", f) for f in os.listdir(os.path.join(src_root, "archive (6)/fire_dataset/fire_images"))]
    random.shuffle(fire_imgs)
    split = int(train_ratio * len(fire_imgs))
    for i, path in enumerate(fire_imgs):
        phase = "train" if i < split else "val"
        shutil.copy(path, os.path.join(dst_root, phase, "fire", f"fire_{i}.jpg"))

    # 2. Road raw images (label as crack or pothole)
    road_root = os.path.join(src_root, "dataset/road/Dataset")
    road_folders = [os.path.join(road_root, d) for d in os.listdir(road_root) if os.path.isdir(os.path.join(road_root, d))]

    crack_imgs, pothole_imgs, road_imgs = [], [], []
    # print((road_folders))
    for folder in road_folders:
        foldername = folder.split('\\')[-1]
        # print(foldername)
        raw_path = os.path.join(folder, f"{foldername}_RAW.jpg")
        crack_mask = os.path.join(folder, f"{foldername}_CRACK.png")
        pothole_mask = os.path.join(folder, f"{foldername}_POTHOLE.png")
        print(mask_has_content(crack_mask))
        if mask_has_content(crack_mask) or mask_has_content(pothole_mask):
            road_imgs.append(raw_path)
        # Ignore raw.jpg with no valid mask
    print(pothole_imgs, crack_imgs)
    # Shuffle and split crack
    random.shuffle(road_imgs)
    split = int(train_ratio * len(road_imgs))
    for i, path in enumerate(road_imgs):
        phase = "train" if i < split else "val"
        shutil.copy(path, os.path.join(dst_root, phase, "road", f"road{i}.jpg"))

    # Shuffle and split crack
    # random.shuffle(crack_imgs)
    # split = int(train_ratio * len(crack_imgs))
    # for i, path in enumerate(crack_imgs):
    #     phase = "train" if i < split else "val"
    #     shutil.copy(path, os.path.join(dst_root, phase, "crack", f"crack_{i}.jpg"))
    # 
    # # Shuffle and split pothole
    # random.shuffle(pothole_imgs)
    # split = int(train_ratio * len(pothole_imgs))
    # for i, path in enumerate(pothole_imgs):
    #     phase = "train" if i < split else "val"
    #     shutil.copy(path, os.path.join(dst_root, phase, "pothole", f"pothole_{i}.jpg"))

    print("✅ Dataset reorganized and ready for training!")

prepare_classification_dataset()


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

In [46]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import numpy as np

# Set paths
data_dir = "data_classification"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

# Image settings
img_size = (224, 224)
batch_size = 32

# Data Augmentation + Normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=10
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Save class names for reporting later
class_names = list(train_generator.class_indices.keys())
print("Classes:", class_names)

# Load Pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base
base_model.trainable = False

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
# output = Dense(3, activation='softmax')(x)
output = Dense(2, activation='softmax')(x)
# 
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate
val_generator.reset()
preds = model.predict(val_generator, verbose=1)
pred_labels = np.argmax(preds, axis=1)  
true_labels = val_generator.classes



Found 2390 images belonging to 2 classes.
Found 598 images belonging to 2 classes.
Classes: ['fire', 'road']
Epoch 1/10


d:\Anaconda\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.7164 - loss: 0.5650 - val_accuracy: 0.8161 - val_loss: 0.3514
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 229s 3s/step - accuracy: 0.8578 - loss: 0.3538 - val_accuracy: 0.9231 - val_loss: 0.1950
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 197s 3s/step - accuracy: 0.9166 - loss: 0.2355 - val_accuracy: 0.9465 - val_loss: 0.1377
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 211s 3s/step - accuracy: 0.9402 - loss: 0.1740 - val_accuracy: 0.9381 - val_loss: 0.1391
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.9558 - loss: 0.1357 - val_accuracy: 0.9448 - val_loss: 0.1262
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 217s 3s/step - accuracy: 0.9454 - loss: 0.1525 - val_accuracy: 0.9448 - val_loss: 0.1237
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 241s 3s/step - accuracy: 0.9656 - loss: 0.1078 - val_accuracy: 0.9532 - val_loss: 0.0868
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 242s 3s/step - accuracy: 0.9697 - loss: 0.0937 - val_accuracy: 0.9699 - val_loss: 0.

In [47]:
print(classification_report(true_labels, pred_labels, target_names=class_names, labels=[0, 1, 2]))



              precision    recall  f1-score   support

        fire       0.99      0.97      0.98       151
        road       0.99      1.00      0.99       447

   micro avg       0.99      0.99      0.99       598
   macro avg       0.66      0.66      0.66       598
weighted avg       0.99      0.99      0.99       598



d:\Anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:2326: UserWarning: labels size, 3, does not match size of target_names, 2
  warnings.warn(
d:\Anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio

In [50]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(true_labels, pred_labels),confusion_matrix(true_labels,  pred_labels)

(0.9899665551839465,
 array([[147,   4],
        [  2, 445]], dtype=int64))

In [36]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model, class_names):
    # Load image and resize
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert image to array
    img_array = image.img_to_array(img)
    
    # Normalize the same way as training (rescale to [0, 1])
    img_array /= 255.0
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    print(prediction)
    predicted_class = np.argmax(prediction, axis=1)

    print(f"Predicted Class: {class_names[predicted_class[0]]}")
    print(predicted_class)


In [54]:
predict_image('./data_classification/val/road/road1787.jpg', model, class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[3.7867947e-06 9.9999619e-01]]
Predicted Class: road
[1]


In [63]:
model.save('incident_detection_final_2.keras')

In [70]:
import tensorflow as tf

model = tf.keras.models.load_model('./incident_detection_final_2.keras')

d:\Anaconda\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [67]:
cn = ['fire', 'road']
predict_image('./data_classification/val/road/road1787.jpg', model, cn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
[[3.7867947e-06 9.9999619e-01]]
Predicted Class: road
[1]


In [71]:
model_json = model.to_json()

with open("model_final.json", "w") as json_file:
    json_file.write(model_json)

In [72]:
!pip install tensorflowjs


^C


In [73]:
model.input_shape

(None, 224, 224, 3)

In [ ]:
i